In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM, Dropout,Flatten
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

## Import dataset

In [ ]:
bitDS = pd.read_csv("/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-12-31.csv")


### Have a look on the data

In [ ]:
bitDS.head()

### Convert timestamp to date

In [ ]:
bitDS["Timestamp"] = pd.to_datetime(bitDS["Timestamp"] ,unit="s").dt.date


In [ ]:
bitDS.head()

### Group by date

In [ ]:
group = bitDS.groupby('Timestamp')
ds = group['Weighted_Price'].mean()


In [ ]:
ds.head(10)


### The aim is making prediction of daily Weighted_Price data.


### Split the data

In [ ]:
# last 50 rows as the test data.
train_df = ds.iloc[0: -50]
test_df = ds.iloc[-50:]
print("Nr. of observations in the train set",train_df.shape)
print("Nr. of observations in the test set",test_df.shape)


### Preprocessing the data

In [ ]:
#Convert to 1-dimensional
training_values = train_df.values
training_values = np.reshape(training_values, (len(training_values), 1))

#Scale data
scaler = MinMaxScaler()
training_values = scaler.fit_transform(training_values)
x_train = training_values[0: len(training_values)-1]
y_train = training_values[1: len(training_values)]
x_train = np.reshape(x_train, (len(x_train), 1, 1))


### Define the LSTM model

In [ ]:
model = Sequential()
model.add(LSTM(10,input_shape = (None,1), activation="relu", return_sequences=True))

model.add(Dense(1))



### Compile the model

In [ ]:
model.compile(loss="mean_squared_error",optimizer="adam")



### Train the model

In [ ]:
y_train.shape

In [ ]:
model.fit(x_train,y_train,epochs=50,batch_size=32)

### Making the predictions


In [ ]:
# Convert to 1-dimensional
test_values = test_df.values
test_values = np.reshape(test_values, (len(test_values), 1))
test_values = scaler.transform(test_values)
test_values = np.reshape(test_values, (len(test_values), 1, 1))
predicted_price = model.predict(test_values)
predicted_price = np.reshape(predicted_price, (len(predicted_price), 1))
predicted_price = scaler.inverse_transform(predicted_price)


### Vizualize the results

In [ ]:
plt.figure(figsize=(15,5))
ax = plt.gca()  
plt.plot(test_df.values, color = 'red', label = 'Real Price')
plt.plot(predicted_price, color = 'green', label = 'Predicted Price')
plt.title('BTC Price Prediction')
test_df = test_df.reset_index()
x=test_df.index
labels = test_df['Timestamp']
plt.xticks(x, labels, rotation = 'vertical')
plt.xlabel('Time')
plt.ylabel('BTC Price(USD)')
plt.legend(loc=4, prop={'size': 14})
plt.show()